# Part 3 - Repeated Measures Analysis

- Does the relationship between sleep duration and cognitive performance differ between domains?

In [1]:
# Import all required Python modules
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels.sandbox.regression.predstd import wls_prediction_std
from scipy import stats, linalg

import sys
sys.path.insert(0, '../lib')
import sleep_study_utils as ss

%matplotlib inline
idx = pd.IndexSlice

/home/conor/.virtualenvs/cbs/lib/python3.5/site-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


In [2]:
# List of all columns in the data frame that have scores
score_columns = ss.score_columns() + [score+"_score" for score in ss.COMPOSITE_SCORE_NAMES]
domain_names  = [domain+"_score" for domain in ss.FACTOR_NAMES]

In [3]:
# Load the final data sample (saved in Part 1)
data = pd.read_pickle('../data/final_sample.pickle.bz2')

In [4]:
# Shift (offset) continuous predictor variables so that regression 
# intercepts are interpretable. (mean centre variables)
age_offset   = data.loc[:,'age_at_test'].mean()
sleep_offset = data.loc[:,'typical_sleep_duration'].mean()
data.loc[:,'age_at_test'] -= age_offset
data.loc[:,'typical_sleep_duration'] -= sleep_offset
data.loc[:,'prev_night_sleep_duration'] -= sleep_offset

In [5]:
# Create a stacked dataframe, where every subject has three rows (one per domain)
stacked_data = pd.DataFrame(columns = ss.QUESTIONNAIRE_ITEMS+['score'])
sample_data  = data.sample(frac=1.0, replace=False)
for score in domain_names:
    sample_data['score_type'] = score[:-6]
    score_data = sample_data[ss.QUESTIONNAIRE_ITEMS+[score,'score_type']].set_index('score_type', append=True).copy()
    score_data.rename(columns={score:'score'}, inplace=True)
    score_data['score_type'] = score_data.index.get_level_values('score_type') 
    score_data['subject'] = sample_data.index
    stacked_data = pd.concat([stacked_data, score_data])
stacked_data['score_type'] = stacked_data['score_type'].astype('category')
stacked_data['subject'] = stacked_data['subject'].astype('category')
stacked_data.head()

,age_at_test,anxiety,depression,education,gender,prev_night_sleep_duration,score,score_type,subject,typical_sleep_duration
"(46640, STM)",-13.689142,Not during the past month,Not during the past month,High School Diploma,Male,1.590364,0.021625,STM,46640.0,0.590364
"(23579, STM)",-15.689142,Not during the past month,Not during the past month,Bachelor's Degree,Male,0.840364,0.878897,STM,23579.0,0.590364
"(29132, STM)",14.310858,Not during the past month,Not during the past month,Master's Degree,Female,1.090364,-0.184048,STM,29132.0,-0.409636
"(56835, STM)",-9.689142,Less than once a week,Less than once a week,High School Diploma,Male,-1.659636,0.825750,STM,56835.0,-0.409636
"(35801, STM)",14.310858,Not during the past month,Not during the past month,High School Diploma,Male,1.090364,-0.673336,STM,35801.0,-0.409636


In [6]:
stacked_data.shape

(32658, 10)

In [7]:
# Build all the  terms that will go into the fixed-effects regression formula
# Include all interactions with cognitive domain, given that Hampshire et al. (2012)
# show that these factors affect the domains differently.
age_regressors   = set(['age_at_test'])
sleep_regressors = set(['np.power(typical_sleep_duration, 2)', 'typical_sleep_duration'])
other_covariates = set(['gender', 'education', 'anxiety', 'depression'])
domain_regressor = set(['score_type'])
age_by_sleep     = ss.build_interaction_terms(age_regressors, sleep_regressors)
sleep_by_domain  = ss.build_interaction_terms(sleep_regressors, domain_regressor)
age_by_domain    = ss.build_interaction_terms(age_regressors, domain_regressor)
other_by_domain  = ss.build_interaction_terms(other_covariates, domain_regressor)
mixed_fx_factors = age_regressors | sleep_regressors | domain_regressor | sleep_by_domain | age_by_domain | age_by_sleep | other_covariates | other_by_domain

In [8]:
# Fit the model, using a random-intercepts model grouping by subject
# That is, each subject gets their own intercept.
mixed_fx_model   = smf.mixedlm(ss.build_model_expression(mixed_fx_factors)%'score', stacked_data, groups=stacked_data["subject"] )
mixed_fx_result  = mixed_fx_model.fit(reml=False)

/home/conor/.virtualenvs/cbs/lib/python3.5/site-packages/statsmodels/regression/mixed_linear_model.py:2019: ConvergenceWarning: The MLE may be on the boundary of the parameter space.
  warnings.warn(msg, ConvergenceWarning)


In [9]:
mixed_fx_result.summary()

<class 'statsmodels.iolib.summary2.Summary'>
"""
                                    Mixed Linear Model Regression Results
=============================================================================================================
Model:                              MixedLM                  Dependent Variable:                  score      
No. Observations:                   32658                    Method:                              ML         
No. Groups:                         10886                    Scale:                               0.8638     
Min. group size:                    3                        Likelihood:                          -43948.3726
Max. group size:                    3                        Converged:                           Yes        
Mean group size:                    3.0                                                                      
-------------------------------------------------------------------------------------------------------------
                                                                  Coef.  Std.Err.    z    P>|z| [0.025 0.975]
-------------------------------------------------------------------------------------------------------------
Intercept                                                          0.041    0.043   0.964 0.335 -0.043  0.125
education[T.Doctoral or Professional Degree]                       0.037    0.031   1.195 0.232 -0.024  0.098
education[T.High School Diploma]                                  -0.116    0.023  -5.026 0.000 -0.161 -0.071
education[T.Master's Degree]                                       0.006    0.024   0.252 0.801 -0.040  0.052
education[T.None]                                                 -0.127    0.062  -2.042 0.041 -0.249 -0.005
score_type[T.STM]                                                 -0.276    0.061  -4.544 0.000 -0.395 -0.157
score_type[T.Verbal]                                               0.107    0.061   1.758 0.079 -0.012  0.226
depression[T.Less than once a week]                               -0.018    0.046  -0.395 0.693 -0.107  0.071
depression[T.Not during the past month]                           -0.027    0.046  -0.587 0.558 -0.116  0.063
depression[T.Once or twice a week]                                -0.019    0.046  -0.403 0.687 -0.109  0.072
depression[T.Three or more times a week]                          -0.092    0.049  -1.889 0.059 -0.188  0.003
gender[T.Male]                                                     0.014    0.019   0.740 0.459 -0.023  0.051
gender[T.Other]                                                    0.177    0.108   1.647 0.100 -0.034  0.389
anxiety[T.Less than once a week]                                   0.021    0.039   0.522 0.601 -0.056  0.097
anxiety[T.Not during the past month]                               0.056    0.039   1.441 0.149 -0.020  0.133
anxiety[T.Once or twice a week]                                    0.035    0.039   0.888 0.375 -0.042  0.112
anxiety[T.Three or more times a week]                              0.045    0.041   1.105 0.269 -0.035  0.124
depression[T.Less than once a week]:score_type[T.STM]              0.022    0.065   0.336 0.737 -0.105  0.148
depression[T.Not during the past month]:score_type[T.STM]          0.045    0.065   0.700 0.484 -0.081  0.172
depression[T.Once or twice a week]:score_type[T.STM]               0.032    0.065   0.490 0.624 -0.096  0.159
depression[T.Three or more times a week]:score_type[T.STM]         0.101    0.069   1.462 0.144 -0.034  0.236
depression[T.Less than once a week]:score_type[T.Verbal]          -0.010    0.065  -0.161 0.872 -0.137  0.116
depression[T.Not during the past month]:score_type[T.Verbal]      -0.070    0.065  -1.080 0.280 -0.196  0.057
depression[T.Once or twice a week]:score_type[T.Verbal]           -0.020    0.065  -0.302 0.763 -0.147  0.108
depression[T.Three or more times a week]:score_type[T.Verbal]      0.082    0.069   1.192 0.233 -0.053  0.217
anxiety[T.Less than once a 

In [10]:
# Test the overall interaction
mixed_fx_result.f_test("""
    np.power(typical_sleep_duration, 2):score_type[T.STM] = 
    np.power(typical_sleep_duration, 2):score_type[T.Verbal] = 
    typical_sleep_duration:score_type[T.STM] = 
    typical_sleep_duration:score_type[T.Verbal] = 0
""")

<class 'statsmodels.stats.contrast.ContrastResults'>
<F test: F=array([[4.06334507]]), p=0.0027016143905193033, df_denom=32602, df_num=4>

In [11]:
# Contrast STM to Reasoning (baseline)
mixed_fx_result.f_test("""
    np.power(typical_sleep_duration, 2):score_type[T.STM] = 
    typical_sleep_duration:score_type[T.STM] = 0
""")

<class 'statsmodels.stats.contrast.ContrastResults'>
<F test: F=array([[5.90354059]]), p=0.002732681712589975, df_denom=32602, df_num=2>

In [12]:
# Contrast Verbal to Reasoning (baseline)
mixed_fx_result.f_test("""
    np.power(typical_sleep_duration, 2):score_type[T.Verbal] = 
    typical_sleep_duration:score_type[T.Verbal] = 0
""")

<class 'statsmodels.stats.contrast.ContrastResults'>
<F test: F=array([[0.75617251]]), p=0.4694680811257468, df_denom=32602, df_num=2>

In [13]:
# Contrast Verbal and STM
mixed_fx_result.f_test("""
    np.power(typical_sleep_duration, 2):score_type[T.STM] - np.power(typical_sleep_duration, 2):score_type[T.Verbal] = 
    typical_sleep_duration:score_type[T.STM] - typical_sleep_duration:score_type[T.Verbal] = 0
""")

<class 'statsmodels.stats.contrast.ContrastResults'>
<F test: F=array([[5.53032211]]), p=0.00396843207043223, df_denom=32602, df_num=2>